In [157]:
# IMPORT LIBRARIES
#import psycopg2 as pg2
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import snowflake.connector

In [158]:
#CONNECT TO SNOWFLAKE WITH REPORTER CREDENTIALS
conn = snowflake.connector.connect(
    user='emirkabasoglu', #emirkabasoglu
    password='7Bb13287', #7Bb13287
    account='se63477.north-europe.azure',
    warehouse='TRANSFORMING_WH',
    database='PROD_ANALYTICS_DB'
    )

In [166]:
#START THE CURSOR AND RUN THE QUERY
cur = conn.cursor()
cur.execute('''

SELECT 
t4.customer_name,
t1.customer_id,t1.subscription_id,t1.status,t1.quantity,t1.arr_eur,t1.start_date,t1.end_date,t2.external_name,
DATEDIFF('YEARS',START_DATE,END_DATE) AS Contract_length,t3.billing_period_unit,
t3.billing_period_frequency,
case when start_date >= '2024-01-01' then 'New Sales' else 'Renewal' end as sales_type
FROM DBT_DEPLOY.SUBSCRIPTIONS t1 
left join DBT_DEPLOY.products t2 on t1.item_id=t2.item_id
left join DBT_DEPLOY.stg_chargebee__subscription t3 on t1.subscription_id=t3.subscription_id
left join DBT_DEPLOY.customers t4 on t4.customer_id=t1.customer_id
where (item_family_id IN ('3','4') OR t1.item_id IN ('8_2','8_3'))
AND (end_date >'2023-12-31') and (end_date!='2024-12-31') and t1.status!='cancelled'
and start_date < '2025-01-01'
order by end_date 
    '''
)


In [167]:
# ADD DESCRIPTION AS COLUMN LABELS
items_db_fetched = cur.fetchall()
cols = []
for i in cur.description:
    cols.append(i[0])

df_items_db_fetched = pd.DataFrame(items_db_fetched, columns=cols)

In [168]:
# ASSIGN TO MAIN DF FOR FUTHER TRANSFORMATION
df_main = df_items_db_fetched

In [169]:
df_main.dtypes

CUSTOMER_NAME                object
CUSTOMER_ID                  object
SUBSCRIPTION_ID              object
STATUS                       object
QUANTITY                      int64
ARR_EUR                     float64
START_DATE                   object
END_DATE                     object
EXTERNAL_NAME                object
CONTRACT_LENGTH               int64
BILLING_PERIOD_UNIT          object
BILLING_PERIOD_FREQUENCY      int64
SALES_TYPE                   object
dtype: object

In [170]:
#CONVERT ALL THE COLUMN NAMES LOWER CASE

df_main.columns= df_main.columns.str.lower()


In [164]:
#DROP ANY COLUMN YOU WANT
#df_main = df_main.drop(columns=['_fivetran_synced'])


In [171]:
# OUTPUT FINAL FILE WITH ALL DIFFERENCES
df_main.to_excel('/Users/emirkabasoglu/Desktop/Eloomi/VS Codes/Jupyter Notebook Scripts/Snowflake Scripts/snowflake-to-excel.xlsx')